# 원불교 교전 데이터 대상 BertWordPieceTokenizer 훈련

## 1. 개요

원불교 교전 데이터를 대상으로 토크나이저를 훈련합니다.

* 데이터셋:
  * 원불교 교전
* 토크나이저:
  * 훈련: 허깅 페이스 [Tokenizers](https://huggingface.co/docs/tokenizers/index) 라이브러리의 [BertWordPieceTokenizer](https://github.com/huggingface/tokenizers/blob/main/bindings/python/py_src/tokenizers/implementations/bert_wordpiece.py) 클래스
  * 사용: 허깅 페이스 [Transformers](https://huggingface.co/docs/transformers/index) 라이브러리의 [AutoTokenizer](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer) 클래스

## 2. 말뭉치 준비

In [1]:
corpus_file = "./won/data/won04-gyojeon.txt"

말뭉치 파일에 대하여 아래 정보를 출력합니다.

* 총 줄 수
* 총 단어 수(단어를 공백으로 구분할 때)
* 가장 많은 단어를 포함하고 있는 줄의 단어 수
* 가장 많은 단어를 포함하고 있는 줄의 텍스트

In [2]:
def count_lines_and_words(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            line_count = len(lines)
            word_count = 0
            
            max_line_pos = 0
            max_line_length = 0
            max_line_text = ""
            
            for i, line in enumerate(lines):
                words = line.split() # 공백으로 단어 분리
                line_length = len(words)
                word_count += line_length
                
                if line_length > max_line_length:
                    max_line_pos = i
                    max_line_length = line_length
                    max_line_text = line
            return line_count, word_count, max_line_pos, max_line_length, max_line_text
    except FileNotFoundError:
        print(f"오류: 파일 '{filepath}'를 찾을 수 없습니다.")
        return None, None, None, None
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        return None, None, None, None

# 함수 호출 및 결과 출력
lines, words, max_line_pos, max_line_length, max_line_text = count_lines_and_words(corpus_file)

if lines is not None and words is not None:
    print(f"===== 파일 분석 결과 ({corpus_file}) =====")
    print(f">> 총 줄 수  : {lines:8,}")
    print(f">> 총 단어 수: {words:8,}")
    print(">> 가장 많은 단어를 포함하고 있는 줄:")
    print(f"* 줄 위치: {max_line_pos:8,}")
    print(f"* 단어 수: {max_line_length:8,}")
    print("* 텍스트 :")
    print("-----")
    print(max_line_text)
    print("-----")

===== 파일 분석 결과 (./won/data/won04-gyojeon.txt) =====
>> 총 줄 수  :    1,082
>> 총 단어 수:   59,541
>> 가장 많은 단어를 포함하고 있는 줄:
* 줄 위치:      804
* 단어 수:      362
* 텍스트 :
-----
17.한 사람이 대종사께 뵈옵고 여러 가지로 담화하는 가운데 [전주․이리 사이의 경편철도(輕便鐵道)는 본래 전라도 각지의 부호들이 주식 출자로 경영하는 것이라, 그들은 언제나 그 경편차를 무료로 이용하고 다닌다.] 하면서 매우 부러워하는 태도를 보이거늘, 대종사 말씀하시기를 [그대는 참으로 가난하도다. 아직 그 차 하나를 그대의 소유로 삼지 못하였는가.] 그 사람이 놀라 여쭙기를 [경편차 하나를 소유하자면 상당한 돈이 있어야 할 것이온데 이 같은 무산자로서 어떻게 그것을 소유할 수 있사오리까.] 대종사 말씀하시기를 [그러므로, 그대를 가난한 사람이라 하였으며, 설사 그대가 경편차 하나를 소유하였다 할지라도 나는 그것으로 그대를 부유한 사람이라고는 아니할 것이니, 이제 나의 살림하는 이야기를 좀 들어보라. 나는 저 전주 경편차뿐 아니라 나라 안의 차와 세계의 모든 차까지도 다 내 것을 삼은지가 벌써 오래 되었노니, 그대는 이 소식을 아직도 모르는가.] 그 사람이 더욱 놀라 사뢰기를 [그 말씀은 실로 요량 밖의 교훈이시므로 어리석은 소견으로는 그 뜻을 살피지 못하겠나이다.] 대종사 말씀하시기를 [사람이 기차 하나를 자기의 소유로 하려면 거액(巨額)의 자금이 일시에 들어야 할 것이요, 운영하는 모든 책임을 직접 담당하여 많은 괴로움을 받아야 할 것이나, 나의 소유하는 법은 그와 달라서 단번에 거액을 들이지도 아니하며, 모든 운영의 책임을 직접 지지도 아니하고, 다만 어디를 가게 되면 그 때마다 얼마씩의 요금만 지불하고 나의 마음대로 이용하는 것이니, 주야로 쉬지 않고 우리 차를 운전하며, 우리 철도를 수선하며, 우리 사무를 관리하여 주는 모든 우리 일꾼들의 급료와 비용이 너무

## 3. 토크나이저 훈련

`BertWordPieceTokenizer`로 훈련합니다.

In [3]:
import os
from tokenizers import BertWordPieceTokenizer

data_files = [
    corpus_file
]

vocab_size = 7000
limit_alphabet = 6000
min_frequency = 5

# 토크나이저 객체 생성
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True, # False로 하면 한글 단어가 [UNK]로 토큰화됨
    lowercase=False,
)

# 훈련
tokenizer.train(
    data_files,
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=limit_alphabet,
    wordpieces_prefix="##",
)

토큰화 결과에 `[CLS]`, `[SEP]` 토큰을 추가하는 후처리 템플릿을 지정합니다.

In [4]:
from tokenizers.processors import TemplateProcessing

# 후처리 템플릿 지정 - 토큰화 결과에 [CLS], [SEP] 토큰 추가
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)

훈련 결과를 `tokenizer.json` 파일과 `vocab.txt` 파일에 저장합니다.

In [5]:
# 파일 저장
output_dir = "./won/tokenizers"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

tokenizer_dir = os.path.join(output_dir, f"bert_word_piece_tokenizer_{vocab_size}")
tokenizer_file = os.path.join(tokenizer_dir, "tokenizer.json")
if not os.path.exists(tokenizer_dir):
    os.makedirs(tokenizer_dir)
    
tokenizer.save_model(tokenizer_dir) # vocab.txt 파일 저장
tokenizer.save(tokenizer_file) # tokenizer.json 파일 저장

print(f"Saved tokenizer files to {tokenizer_dir}")

Saved tokenizer files to ./won/tokenizers\bert_word_piece_tokenizer_7000


## 4. 토크나이저 사용

In [6]:
sample_texts_for_test = [
    "물질이 개벽되니 정신을 개벽하자",
    "응용(應用)하는 데 온전한 생각으로 취사하기를 주의할 것이요"
]

In [7]:
import sys, traceback

def tokenize_and_print(tokenizer, texts):
    for text in texts:
        print("#" * 80)
        print("TEXT: '" + text + "'")
        try:
            print("===== tokenizer.tokenize() =====")
            tokenized = tokenizer.tokenize(text)
            print(tokenized)

            print("===== tokenizer.encode() =====")
            encoded = tokenizer.encode(text)
            print(encoded)
            print(tokenizer.convert_ids_to_tokens(encoded))

            print("===== tokenizer() =====")
            inputs = tokenizer(text)
            print(inputs)
        except Exception as e:
            traceback.print_exc(file=sys.stdout)

In [8]:
from transformers import AutoTokenizer, BertTokenizerFast, DistilBertTokenizerFast

tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)
tokenize_and_print(tokenizer, sample_texts_for_test)

# AutoTokenizer와 동일한 결과를 얻을 수 있음
#tokenizer = BertTokenizerFast.from_pretrained(tokenizer_dir)
#tokenize_and_print(tokenizer, sample_texts_for_test)

# AutoTokenizer와 동일한 결과를 얻을 수 있음
#tokenizer = DistilBertTokenizerFast.from_pretrained(tokenizer_dir)
#tokenize_and_print(tokenizer, sample_texts_for_test)

################################################################################
TEXT: '물질이 개벽되니 정신을 개벽하자'
===== tokenizer.tokenize() =====
['물질이', '개벽', '##되', '##니', '정신을', '개벽', '##하', '##자']
===== tokenizer.encode() =====
[2, 4592, 5197, 1204, 1087, 1793, 5197, 1076, 1141, 3]
['[CLS]', '물질이', '개벽', '##되', '##니', '정신을', '개벽', '##하', '##자', '[SEP]']
===== tokenizer() =====
{'input_ids': [2, 4592, 5197, 1204, 1087, 1793, 5197, 1076, 1141, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
################################################################################
TEXT: '응용(應用)하는 데 온전한 생각으로 취사하기를 주의할 것이요'
===== tokenizer.tokenize() =====
['응용', '(', '應', '用', ')', '하는', '데', '온전한', '생각으로', '취사', '##하기를', '주의할', '것이요']
===== tokenizer.encode() =====
[2, 2239, 7, 442, 632, 8, 1241, 1343, 4070, 3551, 1917, 2128, 2568, 1242, 3]
['[CLS]', '응용', '(', '應', 

두 개의 텍스트를 하나의 쌍으로 토큰화합니다.

In [9]:
print("===== tokenizer.tokenize() =====")
tokenized = tokenizer.tokenize(sample_texts_for_test[0], sample_texts_for_test[1])
print(tokenized)

print("===== tokenizer.encode() =====")
encoded = tokenizer.encode(sample_texts_for_test[0], sample_texts_for_test[1])
print(encoded)
print(tokenizer.convert_ids_to_tokens(encoded))

print("===== tokenizer() =====")
inputs = tokenizer(sample_texts_for_test[0], sample_texts_for_test[1])
print(inputs)
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))

===== tokenizer.tokenize() =====
['물질이', '개벽', '##되', '##니', '정신을', '개벽', '##하', '##자', '응용', '(', '應', '用', ')', '하는', '데', '온전한', '생각으로', '취사', '##하기를', '주의할', '것이요']
===== tokenizer.encode() =====
[2, 4592, 5197, 1204, 1087, 1793, 5197, 1076, 1141, 3, 2239, 7, 442, 632, 8, 1241, 1343, 4070, 3551, 1917, 2128, 2568, 1242, 3]
['[CLS]', '물질이', '개벽', '##되', '##니', '정신을', '개벽', '##하', '##자', '[SEP]', '응용', '(', '應', '用', ')', '하는', '데', '온전한', '생각으로', '취사', '##하기를', '주의할', '것이요', '[SEP]']
===== tokenizer() =====
{'input_ids': [2, 4592, 5197, 1204, 1087, 1793, 5197, 1076, 1141, 3, 2239, 7, 442, 632, 8, 1241, 1343, 4070, 3551, 1917, 2128, 2568, 1242, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', '물질이', '개벽', '##되', '##니', '